https://www.kaggle.com/xhlulu/siim-covid-19-convert-to-jpg-256px

In [1]:
import pandas as pd
import numpy as np
import pydicom
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

import os

In [2]:
df = pd.read_csv('/workspace/data/df_train_study_level.csv')

In [3]:
save_path = "/workspace/data/train_640_3/"
if os.path.exists(save_path) == False:
    os.makedirs(save_path)

In [7]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

from PIL import Image

def save_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)    
    data = data.astype(np.float32)
    data = cv2.resize(data, (640, 640))
    
    png_path = save_path + path.split('/')[-1].replace("dcm", "npy")
    np.save(png_path, data)
    
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

# def save_xray(path, voi_lut = True, fix_monochrome = True):
#     # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
#     dicom = pydicom.read_file(path)
    
#     # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
#     # "human-friendly" view
#     if voi_lut:
#         data = apply_voi_lut(dicom.pixel_array, dicom)
#     else:
#         data = dicom.pixel_array
               
#     # depending on this value, X-ray may look inverted - fix that:
#     if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
#         data = np.amax(data) - data
        
#     data = data - np.min(data)
#     data = data / np.max(data)
# #     data = (data * 255).astype(np.uint8)
#     data = data.astype(np.float32)
    
#     png_path = save_path + path.split('/')[-1].replace("dcm", "npy")
#     np.save(png_path, data)

In [8]:
dicom_path_list = df.dicom_path

In [9]:
from multiprocessing import Pool
import time

pool = Pool(processes=20)

with tqdm(total=len(dicom_path_list)) as t:
    for _ in pool.imap_unordered(save_xray, dicom_path_list):
        t.update(1)

  2%|▏         | 150/6334 [00:09<05:57, 17.31it/s]/opt/conda/lib/python3.8/site-packages/pydicom/pixel_data_handlers/numpy_handler.py:341: UserWarning: The length of the pixel data in the dataset (13262360 bytes) indicates it contains excess padding. 216296 bytes will be removed from the end of the data
  warnings.warn(msg)
100%|██████████| 6334/6334 [04:45<00:00, 22.20it/s]


In [39]:
df["npy_path"] = [save_path + path.split('/')[-1].replace("dcm", "npy") for path in df["dicom_path"]]

In [40]:
df_latest = pd.read_csv("/workspace/data/df_train_study_level_npy640_3_w_bbox.csv")

In [41]:
npy_path = []
for i in range(len(df_latest)):
    path = df_latest.iloc[i].npy_path.replace("640_2", "640_3").replace('png', 'npy')
    if os.path.exists(path):
        npy_path.append(path)

In [42]:
df_latest["npy_path"] = npy_path

In [43]:
df_latest

,id,x,y,w,h,study_id,image_id,have_box,dicom_path,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,cv,npy_path,is_none
0,04f41a8958f7_image,688.06282,966.82563,518.48212,1130.17438,6e4a0581cefe,04f41a8958f7,1,/workspace/data/train/6e4a0581cefe/018ed20fa9c...,0,1,0,0,0,/workspace/data/train_640_3/04f41a8958f7.npy,0
1,04f41a8958f7_image,2482.36026,1636.77436,652.47168,652.47168,6e4a0581cefe,04f41a8958f7,1,/workspace/data/train/6e4a0581cefe/018ed20fa9c...,0,1,0,0,0,/workspace/data/train_640_3/04f41a8958f7.npy,0
2,04f41a8958f7_image,1235.67308,1628.03597,509.74353,559.26147,6e4a0581cefe,04f41a8958f7,1,/workspace/data/train/6e4a0581cefe/018ed20fa9c...,0,1,0,0,0,/workspace/data/train_640_3/04f41a8958f7.npy,0
3,0572ef0d0c1a_image,1818.65264,233.50598,613.04395,839.53784,adbfed2da701,0572ef0d0c1a,1,/workspace/data/train/adbfed2da701/e2fa197720c...,0,1,0,0,3,/workspace/data/train_640_3/0572ef0d0c1a.npy,0
4,0572ef0d0c1a_image,598.60492,61.37052,688.54175,881.81674,adbfed2da701,0572ef0d0c1a,1,/workspace/data/train/adbfed2da701/e2fa197720c...,0,1,0,0,3,/workspace/data/train_640_3/0572ef0d0c1a.npy,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9671,ffd9b6cf2961_image,707.25199,722.07926,392.14044,849.18683,7eed9af03814,ffd9b6cf2961,1,/workspace/data/train/7eed9af03814/668a64e3a5f...,0,1,0,0,1,/workspace/data/train_640_3/ffd9b6cf2961.npy,0
9672,ffdc682f7680_image,2729.27083,332.26044,1496.25016,2604.58334,a0cb0b96fb3d,ffdc682f7680,1,/workspace/data/train/a0cb0b96fb3d/ccf363aa080...,0,1,0,0,2,/workspace/data/train_640_3/ffdc682f7680.npy,0
9673,ffdc682f7680_image,1005.81250,1584.67711,662.22913,775.83337,a0cb0b96fb3d,ffdc682f7680,1,/workspace/data/train/a0cb0b96fb3d/ccf363aa080...,0,1,0,0,2,/workspace/data/train_640_3/ffdc682f7680.npy,0
9674,ffe942c8655f_image,208.86463,91.53448,450.96747,628.05473,7d82d53204b8,ffe942c8655f,1,/workspace/data/train/7d82d53204b8/431ee249d16...,0,1,0,0,1,/workspace/data/train_640_3/ffe942c8655f.npy,0


In [44]:
df_latest.to_csv('/workspace/data/df_train_study_level_npy640_4_w_bbox.csv')